In [27]:
import tkinter as tk
from tkinter import filedialog, scrolledtext
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, kendalltau

def load_matrices_from_csv(file_paths):
    matrices = []
    matrix_names = []
    for file_path in file_paths:
        df = pd.read_csv(file_path, header=None)  # 读取CSV文件
        matrices.append(df.values)  # 转换为NumPy数组
        matrix_names.append(file_path.split('/')[-1])  # 提取文件名
    return matrices, matrix_names

def calculate_correlation(matrix_list, method='pearson'):
    num_matrices = len(matrix_list)
    correlations = np.zeros((num_matrices, num_matrices))
    
    for i in range(num_matrices):
        for j in range(i, num_matrices):
            if method == 'pearson':
                corr = np.corrcoef(matrix_list[i].flatten(), matrix_list[j].flatten())[0, 1]
            elif method == 'spearman':
                corr, _ = spearmanr(matrix_list[i].flatten(), matrix_list[j].flatten())
            elif method == 'kendall':
                corr, _ = kendalltau(matrix_list[i].flatten(), matrix_list[j].flatten())
            correlations[i, j] = corr
            correlations[j, i] = corr  # 相关性矩阵是对称的
            
    return correlations

def load_files():
    file_paths = filedialog.askopenfilenames(filetypes=[("CSV files", "*.csv")])
    if file_paths:
        global matrix_list, matrix_names
        matrix_list, matrix_names = load_matrices_from_csv(file_paths)
        matrix_display.config(state=tk.NORMAL)
        matrix_display.delete(1.0, tk.END)  # 清空文本框
        matrix_display.insert(tk.END, "已加载的矩阵:\n")
        for name in matrix_names:
            matrix_display.insert(tk.END, f"- {name}\n")
        matrix_display.config(state=tk.DISABLED)

def calculate_correlation_from_loaded_files(method):
    if matrix_list:
        correlation_matrix = calculate_correlation(matrix_list, method=method)
        matrix_display.config(state=tk.NORMAL)
        matrix_display.delete(1.0, tk.END)  # 清空文本框
        matrix_display.insert(tk.END, f"{method.capitalize()} 相关性矩阵:\n")
        for i, name_i in enumerate(matrix_names):
            for j, name_j in enumerate(matrix_names):
                matrix_display.insert(tk.END, f"({name_i}, {name_j}): {correlation_matrix[i, j]:.4f}\n")
        matrix_display.config(state=tk.DISABLED)
    else:
        matrix_display.config(state=tk.NORMAL)
        matrix_display.delete(1.0, tk.END)
        matrix_display.insert(tk.END, "请先加载矩阵文件")
        matrix_display.config(state=tk.DISABLED)

def close_program():
    root.destroy()

# 创建主窗口
root = tk.Tk()
root.title("矩阵相关性计算")

# 设置窗口大小
root.geometry('600x400')  # 可以根据需要调整窗口大小

# 创建和放置按钮
load_button = tk.Button(root, text="加载CSV文件", command=load_files)
load_button.pack(pady=5)

pearson_button = tk.Button(root, text="计算皮尔逊相关性", command=lambda: calculate_correlation_from_loaded_files('pearson'))
pearson_button.pack(pady=5)

spearman_button = tk.Button(root, text="计算斯皮尔曼相关性", command=lambda: calculate_correlation_from_loaded_files('spearman'))
spearman_button.pack(pady=5)

kendall_button = tk.Button(root, text="计算肯德尔相关性", command=lambda: calculate_correlation_from_loaded_files('kendall'))
kendall_button.pack(pady=5)

close_button = tk.Button(root, text="关闭", command=close_program)
close_button.pack(pady=10)

# 创建文本框用于显示加载的矩阵
matrix_display = scrolledtext.ScrolledText(root, width=70, height=15, wrap=tk.WORD, state=tk.DISABLED)
matrix_display.pack(pady=10)

matrix_list = []
matrix_names = []

# 运行主事件循环
root.mainloop()
